In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt            # library for visualization
from sklearn.model_selection import train_test_split
import functions as fun

[nltk_data] Downloading package punkt to /Users/marta/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/marta/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv("data/DatasetFinal.csv", index_col=0)
df.head(2)

,Course Title,Rating,Level,Schedule,What you will learn,Skill gain,Modules,Instructor,Offered By,Keyword,Course Url,Duration to complete (Approx.),Number of Review
0,Fashion as Design,4.8,Beginner level,Flexible schedule,Not specified,"Art History, Art, History, Creativity","Introduction, Heroes, Silhouettes, Coutures, L...","Anna Burckhardt, Paola Antonelli, Michelle Mil...",The Museum of Modern Art,Arts and Humanities,https://www.coursera.org/learn/fashion-design,20.0,2813
1,Modern American Poetry,4.4,Beginner level,Flexible schedule,Not specified,Not specified,"Orientation, Module 1, Module 2, Module 3, Mod...",Cary Nelson,University of Illinois at Urbana-Champaign,Arts and Humanities,https://www.coursera.org/learn/modern-american...,34.0,100


# 1. PREPROCESSING

In [3]:
# Listados de ciencias y humanidades:
df_science = df[df["Keyword"] != "Arts and Humanities"]
df_humanities = df[df["Keyword"] == "Arts and Humanities"]

row_humanities = list(df_humanities["Course Title"])
row_science = list(df_science["Course Title"][:476])
all_rows = (row_science) + (row_humanities)

print(type(all_rows), len(all_rows))

# Combination of humanities and science labels
y = np.append(np.ones((len(row_science), 1)), np.zeros((len(row_humanities), 1)), axis=0)

<class 'list'> 943


### Train test split: 20% will be in the test set, and 80% in the training set.

In [4]:
# Conjunto de entrenamiento
def get_data_train(title,label):
    # Obtenemos el 80% del conjunto de datos como conjunto de entrenamiento."
    train_x, test_x, train_y, test_y = train_test_split(title, label, test_size=0.20, random_state=42)

    return train_x, test_x, train_y, test_y

train_x, test_x, train_y, test_y = get_data_train(all_rows,y)
print(type(train_x), len(train_x), len(test_x))
print(type(test_y), len(train_y), len(test_y))

<class 'list'> 754 189
<class 'numpy.ndarray'> 754 189


### Word Count - frequency dictionary

In [5]:
# create frequency dictionary
freqs = fun.build_freqs(train_x, train_y)

# check data type
print(f'type(freqs) = {type(freqs)}')

# check length of the dictionary
print(f'len(freqs) = {len(freqs)}')

type(freqs) = <class 'dict'>
len(freqs) = 1359


# 2. Training a Naive Bayes 

In [6]:
logprior, loglikelihood = fun.train_naive_bayes(freqs, train_x, train_y)
print("Logprior = ", logprior)
print("len(loglikelihood) = ", len(loglikelihood))

La cantidad de palabras únicas es: 1202
El número de cursos es:  754
Logprior =  0.031832926573210685
len(loglikelihood) =  1202


In [7]:
loglikelihood.items()

dict_items([('regulatori', 0.6564804038711354), ('商管研究中的賽局分析（二）：資訊經濟學', 0.6564804038711354), ('need', -0.7298139572487552), ('1910', -1.1352790653569196), ('活用希臘哲學', -0.7298139572487552), ('eye', -0.7298139572487552), ('news', -0.7298139572487552), ('implement', 0.6564804038711354), ('explor', -1.828426245916865), ('action', -1.1352790653569196), ('hi', -1.4229611378087006), ('concept', -0.32434884914059076), ('classroom', 0.6564804038711354), ('align', 0.6564804038711354), ('song', -0.7298139572487552), ('introduc', 0.6564804038711354), ('content', -0.036666776688809924), ('mondial', -0.7298139572487552), ('christian', -1.4229611378087006), ('partner', 0.6564804038711354), ('repeat', 0.6564804038711354), ('palatino', -0.7298139572487552), ('emot', -0.7298139572487552), ('korean', -1.4229611378087006), ('in', -1.1352790653569196), ('document', -1.1352790653569196), ('egypt', -1.1352790653569196), ('aleatoriedad', -0.7298139572487552), ('statement', 0.6564804038711354), ('dream', -1.422

## 3 - Test Naive Bayes

In [8]:
new_course = "Sports Business Analysis"
#new_course = "Children in Primary Education" 
#new_course = "Writing the best essay for university interviews about education"

fun.naive_bayes_predict(new_course, logprior, loglikelihood)

7.343420074451084

In [ ]:
def test_naive_bayes(test_x, test_y, logprior, loglikelihood, naive_bayes_predict=fun.naive_bayes_predict):
    """
    Input:
        test_x: A list of titles
        test_y: the corresponding labels for the list of titles
        logprior: the logprior
        loglikelihood: a dictionary with the loglikelihoods for each word
    Output:
        accuracy: (# of titles classified correctly)/(total # of titles)
    """
    accuracy = 0  # return this properly
    y_hats = []
    for title in test_x:
        # if the prediction is > 0
        if naive_bayes_predict(title, logprior, loglikelihood) > 0:
            # the predicted class is 1
            y_hat_i = 1
        else:
            y_hat_i = 0
        # append the predicted class to the list y_hats
        y_hats.append(y_hat_i)

    # error is the average of the absolute values of the differences between y_hats and test_y
    error = np.mean(np.absolute(y_hats - test_y))

    # Accuracy is 1 minus the error
    accuracy = 1 - error

    return accuracy

print("Naive Bayes accuracy = %0.4f" %
      (test_naive_bayes(test_x, test_y, logprior, loglikelihood)))